In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
import subprocess
import inspect
import pathlib
from pathlib import Path

from pyphocorehelpers.Filesystem.source_code_helpers import find_py_files
from pyphocorehelpers.Filesystem.generate_code_index import build_code_index

Automatic pdb calling has been turned OFF


In [12]:
project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy", project_name='NeuroPy')
# project_path, project_db_file_path, tags_file_path, cscope_out_file_path, cscope_files_file_path = project_outputs # unpack



project_name: NeuroPy
	found 100 .py files in project.


In [13]:
project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src", project_name='pyPhoCoreHelpers')

project_name: pyPhoCoreHelpers
	found 57 .py files in project.


In [14]:
project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"], project_name='pyPhoPlaceCellAnalysis')

project_name: pyPhoPlaceCellAnalysis
	found 232 .py files in project.


In [3]:
import glob
import importlib
import inspect
import os

current_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)))
current_module_name = os.path.splitext(os.path.basename(current_dir))[0]
for file in glob.glob(current_dir + "/*.py"):
     name = os.path.splitext(os.path.basename(file))[0]

     # Ignore __ files
     if name.startswith("__"):
         continue
     module = importlib.import_module("." + name,package=current_module_name)

     for member in dir(module):
         handler_class = getattr(module, member)

         if handler_class and inspect.isclass(handler_class):
             print(member)
             


NameError: name '__file__' is not defined

In [ ]:


classes = [cls_name for cls_name, cls_obj in inspect.getmembers(sys.modules['test_cls']) if inspect.isclass(cls_obj)]
print(classes)


# Pre 2023-07-28 Testing

In [ ]:
root_project_path = pathlib.Path("C:/Users/pho/repos/Spike3DWorkEnv/")
cscope_files = root_project_path / "cscope.files"
cscope_out = root_project_path / "cscope.out"
tags = root_project_path / "tags"
db_path = root_project_path / "myproject.db"

repos_source_paths = ["C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src"]
repos_exclude_dir_lists = [None, None, ["pyphoplacecellanalysis/External"]]

all_found_files = []
for a_src_path, exclude_dirs in zip(repos_source_paths, repos_exclude_dir_lists):
    all_found_files.extend(find_py_files(a_src_path, exclude_dirs = exclude_dirs))

# all_found_files

# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])



In [ ]:
# Write the file paths to cscope.files
with open(cscope_files, "w") as f:
	print(f'writing to {cscope_files}...')
	for file_path in all_found_files:
		f.write(str(file_path) + "\n")
	print('\tdone.')

# Old Full Implementation

In [ ]:

def build_code_index(project_path, exclude_dirs=[]):
    project_path = pathlib.Path(project_path)
    cscope_files = project_path / "cscope.files"
    cscope_out = project_path / "cscope.out"
    tags = project_path / "tags"
    db_path = project_path / "myproject.db"

    # Find all .py files in the project directory and its subdirectories
    py_files = project_path.glob("**/*.py")
    py_files = [file_path for file_path in py_files] # to list

    excluded_py_files = []
    if exclude_dirs is not None:
        # Find all .py files in the project directory and its subdirectories, excluding the 'my_exclude_dir' directory
        exclude_paths = [project_path.joinpath(a_dir) for a_dir in exclude_dirs]
        for an_exclude_path in exclude_paths:
            excluded_py_files.extend([file_path for file_path in an_exclude_path.glob("**/*.py")])

    included_py_files = [x for x in py_files if x not in excluded_py_files]

    # Write the file paths to cscope.files
    with open(project_path / "cscope.files", "w") as f:
        for file_path in included_py_files:
            f.write(str(file_path) + "\n")

    # Generate the cscope index
    subprocess.run(["pycscope", "-i", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the ctags index
    subprocess.run(["ctags", "--fields=+i", "-n", "-L", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the CodeQL database
    subprocess.run(["cqmakedb", "-s", str(db_path), "-c", str(cscope_out), "-t", str(tags), "-p"], cwd=str(project_path), check=True)



In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")

In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")

In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])